## Amadeus Data Science Challenge

## Ejercicio 1. Cuenta el número de línease del documento "bookings.csv" 
- Esta situado en: home/dsc/Data/challenge

- Posibles formas de solucionarlo:
    - Usando la línea de comandos
    - Usando bucles en Python
    - Usando Pandas

### Solución 1. Usando la línea de comandos

In [12]:
! pwd

/home/dsc/Data/challenge


In [13]:
%%bash
cd /home/dsc/Data/challenge

In [14]:
ls -l

total 1019072
-rw-rw-r-- 1 dsc dsc 554970628 abr  2  2018 bookings.csv.bz2
-rw-rw-r-- 1 dsc dsc   4232732 ene 27 23:35 bookings_sample.csv
-rw-rw-r-- 1 dsc dsc    535893 ene 25 09:41 bookings_sample.csv.bz2
-rw-rw-r-- 1 dsc dsc 483188920 abr  2  2018 searches.csv.bz2
-rw-rw-r-- 1 dsc dsc     77878 ene 25 13:11 top_airports.csv
-rw-rw-r-- 1 dsc dsc    246473 ene 25 13:35 Untitled1.ipynb
-rw-rw-r-- 1 dsc dsc    248766 ene 25 13:45 Untitled1_por_si_acasov1.ipynb
-rw-rw-r-- 1 dsc dsc        72 ene 25 09:16 Untitled.ipynb


- Vemos que es muy pesado (+5GB), por lo que es mejor trabajar con él en formato bz2
-  Se podría descomprimir con `! bunzip2` y comprimir con `! bzip2`, pero mejor trabajar con este formato bzip
    - Este formato es aconsejable para archivos de +128MB
    - Es más recomendable que .zip, pero más lento. Además, con .bzip2 nos aseguramos que cada uno de ellos solo contenga un archivo, algo que no pasa con los .zip

In [4]:
# El archivo es muy pesado, durará un rato
! bzcat bookings.csv.bz2 | wc -l

bzcat: Can't open input file bookings.csv.bz2: No such file or directory.
0


In [15]:
# Para ejercicios posteriores, vamos a exportar una muestra de las 10.000 primeras líneas
! bzcat ./bookings.csv.bz2 | head -10000 > ./bookings_sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./bookings.csv.bz2, output file = (stdout)


In [10]:
# La convertimos a bz2 para trabjar con todos los archivos en el mismo formato y luego evitar estar cambiando el código
! bzip2 bookings_sample.csv

bzip2: Can't open input file bookins_sample.csv: No such file or directory.


### Solución 2. Usando Python
- Se puede resolver mediante el uso de bucles

In [16]:
import bz2

In [17]:
fileBz2 =  bz2.BZ2File("./bookings_sample.csv.bz2")

In [18]:
type(fileBz2)

bz2.BZ2File

In [19]:
#Para evitar retardos en el cálculo se usará la muestra de 10.000 líneas, pero el concepto queda claro...
k = 0

for line in fileBz2:
    k += 1

print(k)

10000


In [20]:
# Mucho ojo con volver a correr el bucle ahora, ya que dará cero al apuntar ahora el puntero al final...
k = 0

for line in fileBz2:
    k += 1

print(k)

0


### Mejorando la solución 2
- ¿El fichero está cerrado?

In [21]:
fileBz2.closed

False

- Cerramos entonces el fichero

In [22]:
fileBz2.close()

In [23]:
fileBz2.closed

True

### Mejorando aún más la solución 2.
- El `with` está para algo...

In [24]:
import bz2
with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
    k = 0
    for line in fileBz2:
        k += 1
    print(k)

10000


In [ ]:
fileBz2.closed

### Mejorando aún más la solución 2.
- Podemos manejar excepciones con un `try/except`

In [25]:
import bz2
try:
    with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
        k = 0
        for line in fileBz2:
            k += 1
except:
    print("Error counting lines")
        
print(k)

10000


- Podemos dar un paso mas:

In [29]:
import bz2
try:
    with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
        k = 0
        for line in fileBz2:
            k += 1
except FileNotFoundError:
    print("File Not Found")
except:
    print("Unexpected error")
        
print(k)

10000


- Una persona más experimentada hubiera puesto este código, haciendo que salte por defecto el error del mensaje

In [28]:
import bz2
try:
    with bz2.BZ2File("./bookisdfngs_sample.csv.bz2") as fileBz2:
        k = 0
        for line in fileBz2:
            k += 1
except FileNotFoundError as message:
    print(message)
except:
    print("Unexpected error")
        
print(k)

[Errno 2] No such file or directory: './bookisdfngs_sample.csv.bz2'
10000


- O incluso poniendo un `enumerate`

In [30]:
import bz2
try:
    with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
        for k, line in enumerate(fileBz2):
            pass
        print(k+1)
except FileNotFoundError as message:
    print(message)
except ValueError:
    print("value error")
except:
    print('unexpected error')

10000


# Solución 3. Mediante el uso de Pandas

In [31]:
import pandas as pd
! bzcat bookings_sample.csv.bz2 | head -1 |tr ^ "\n"


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings_sample.csv.bz2, output file = (stdout)
act_date           
source
pos_ctry
pos_iata
pos_oid  
rloc          
cre_date           
duration
distance
dep_port
dep_city
dep_ctry
arr_port
arr_city
arr_ctry
lst_port
lst_city
lst_ctry
brd_port
brd_city
brd_ctry
off_port
off_city
off_ctry
mkt_port
mkt_city
mkt_ctry
intl
route          
carrier
bkg_class
cab_class
brd_time           
off_time           
pax
year
month
oid      


- Seleccionando una columna pueod ver el largo, por ejemplo:

In [32]:
df = pd.read_csv("bookings_sample.csv.bz2", sep="^", usecols=["pax"])

In [34]:
df.shape

(9999, 1)

In [35]:
df.shape[0]

9999

- Y si le añado el header, tengo las 10.000 líneas

## Ejercicio 2.

### Calcula los 10 primeros aeropuertos del mundo en 2013 utilizando la librería Pandas y el archivo 'bookings.csv'.
- El aeropuerto de llegada se encuentra en la columna 'arr_port', que contiene el código IATA del aeropuerto.
- Puedes sumar los totales de pasajeros para cada aeropuerto, pero ten en cuenta que hay negativos, correspondientes a las cancelaciones.
- **Bonus Point**. Obtén el nombre real del aeropuerto en base a su código IATA. Se recomienda utilizar la librería GeoBases que se encuentra en GitHub.

#### ¿Cómo resolver el ejercicio?
1. **Familiarízate con los datos**:
    - Piensa qué significa cada columna
    - Qué columnas interesan
    - Si hay NaN y qué hacer con ellos...
    

2. **Crea un plan de acción**.
    - Primero desarrolla un código que funcione con una muestra (Sample test)
    - Ajusta ese código que funciona al big data y testéalo
    - Corre el programa con el big data (Usa los *chunks* en esta transición)

In [1]:
import pandas as pd
import numpy as np

- Comenzamos cogiendo una pequeña muestra para ver la estructura de los datos.
    - Se puede usar `head` o `sample`, si quieremos que la muestra sea aleatoria

In [6]:
b = pd.read_csv("bookings_sample.csv.bz2")
b.head(5)

,act_date ^source^pos_ctry^pos_iata^pos_oid ^rloc ^cre_date ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route ^carrier^bkg_class^cab_class^brd_time ^off_time ^pax^year^month^oid
0,2013-03-05 00:00:00^1A ^DE ^a68dd7ae95...
1,2013-03-26 00:00:00^1A ^US ^e612b9eeee...
2,2013-03-26 00:00:00^1A ^US ^e612b9eeee...
3,2013-03-26 00:00:00^1A ^AU ^0f984b3bb6...
4,2013-03-26 00:00:00^1A ^AU ^0f984b3bb6...


In [11]:
b = pd.read_csv("bookings_sample.csv.bz2", sep = "^")
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
8428,2013-03-20 00:00:00,1S,SA,dfff39b63873ba58a63ce60008b83e28,7f15131373bd047e5a3d61c9cd5370f4,d2601218800a777e76cdff7c3e3ab4f4,2013-03-20 00:00:00,45461,0,JED,...,DELJED,DG,Q,Y,2013-05-02 13:30:00,2013-05-02 16:21:41,1,2013,3,NULL
6322,2013-03-04 00:00:00,1P,US,4804033f441ee30d92980c8b2207e8ae,4868da7a1666bbbc5fe33d377690f8eb,6a810626d8e1fd183e278287bc66f483,2013-01-10 00:00:00,11689,0,DEN,...,TPADEN,FK,S,Y,2013-03-31 18:59:00,2013-03-31 20:37:53,-4,2013,3,NULL
3833,2013-03-13 00:00:00,1A,QA,9e62a8cd8850ea70d0c07647f0f5212a,a938f771b70351d6bfc0cb37bee44616,3a5a99e461a5abc3713f7594e86437c4,2013-03-13 00:00:00,2570,0,AUH,...,AUHDOH,OJ,V,Y,2013-03-17 17:10:00,2013-03-17 17:20:54,1,2013,3,NULL
6257,2013-03-14 00:00:00,1V,US,51de894d954fed791641054a759aeb24,b4afbb99d8f9bb8e68a4e4d397cd85ae,2fa61c9cfaa11da201c50b9d6c35b2b9,2013-03-14 00:00:00,10973,0,FLO,...,MEXGDL,KW,S,Y,2013-04-07 14:00:00,2013-04-07 15:20:23,1,2013,3,NULL
1546,2013-03-21 00:00:00,1V,US,a980a5d457725fed57d2d8f27ec9f432,e4e6eebe01fe70ec6bb27048248b0d44,58ee9c6852513816a39363a1621a0615,2013-03-07 00:00:00,22963,0,SFO,...,HKGTPEHND,XR,Y,Y,2013-03-30 07:30:00,2013-03-30 21:05:54,1,2013,3,NULL


- Comenzamos viendo las dimesiones de nuestra muestra:
    - Tenemos 10.000 filas y 38 columnas

In [8]:
b.shape

(9999, 38)

- Sin embargo, con el `head` no hemos visto todas... Y nos aparecen tres puntos (...) en las columnas indicando que hay más ocultas.
    - Cambiamos las settings de Python para poder verlas todas.

In [12]:
pd.set_option("display.max_columns", None)

In [13]:
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
7437,2013-03-25 00:00:00,1A,SV,3f3e272616951c1785f54af5659334bb,df44a2d505d988426a7611bf575565ae,51dba5a1299d27fe0c953f5c40c4ea55,2013-03-25 00:00:00,6352,0,SAL,SAL,SV,GUA,GUA,GT,SAL,SAL,SV,GUA,GUA,GT,SAL,SAL,SV,GUASAL,GUASAL,GTSV,1,GUASAL,EK,H,Y,2013-04-26 17:37:00,2013-04-26 18:39:20,1,2013,3,NULL
5523,2013-03-12 00:00:00,1A,CO,5fb8d7940a783c062f524aa9dbf77651,d4fa279f8d9bc7f6b8c58e1d5810f8e4,31541c7b0452f67bce13a631a8675a85,2013-03-12 00:00:00,1592,0,BOG,BOG,CO,MDE,MDE,CO,BOG,BOG,CO,BOG,BOG,CO,MDE,MDE,CO,BOGMDE,BOGMDE,COCO,0,BOGMDE,KG,K,Y,2013-03-20 19:26:00,2013-03-20 20:29:11,2,2013,3,BOGOV3262
9076,2013-03-05 00:00:00,1P,LB,fb1e34ecaf06a71d3b38e9da8543f310,24856a47570f69253fd12224fa198cc0,8766ee850e14f10e2311461061462d58,2013-03-05 00:00:00,15082,0,BEY,BEY,LB,CDG,PAR,FR,BEY,BEY,LB,BEY,BEY,LB,CDG,PAR,FR,BEYCDG,BEYPAR,FRLB,1,BEYCDG,KP,T,Y,2013-03-15 10:30:00,2013-03-15 14:02:49,1,2013,3,NULL
7484,2013-03-01 00:00:00,1P,ZA,69ee162f3c4293300ea5fbc0143a3aca,69d84738440ef08ea7f81f553bf8f0ab,f962c8d465bd4165948741a7edc414cd,2013-02-28 00:00:00,81,478,DUR,DUR,ZA,JNB,JNB,ZA,JNB,JNB,ZA,DUR,DUR,ZA,JNB,JNB,ZA,DURJNB,DURJNB,ZAZA,0,DURJNB,DK,D,C,2013-03-01 13:55:00,2013-03-01 15:16:45,-1,2013,3,NULL
6984,2013-03-11 00:00:00,1A,IN,fcbc0d1bcea4d2a58102ec46703205c0,1af805c5dbe50daee200695917e91d5b,ff433decc30088488210aff5f83c4da6,2013-03-11 00:00:00,13646,0,BOM,BOM,IN,LHR,LON,GB,BOM,BOM,IN,VCE,VCE,IT,LHR,LON,GB,LHRVCE,LONVCE,GBIT,1,VCELHR,LK,B,Y,2013-04-19 13:35:00,2013-04-19 14:44:15,1,2013,3,NULL


- Gracias a ello podemos completar la primera parte de nuestro plan y ver qué columnas pueden ser necesarias.

- Los nombres de las columnas son:
    - Acabamos de ver la primera trampa: como se puede ver, algunas columnas tienen nombres con espacios, lo cual al mencionarlas nos puede dar error. 

In [10]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

- Mediante `info` veo el tipo de las columnas, por si hubiera alguna de formato numérico no contabilizada como formato numérico para transformarla mediante `astype`

In [18]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 38 columns):
act_date               9999 non-null object
source                 9999 non-null object
pos_ctry               9999 non-null object
pos_iata               9999 non-null object
pos_oid                9999 non-null object
rloc                   9999 non-null object
cre_date               9999 non-null object
duration               9999 non-null int64
distance               9999 non-null int64
dep_port               9999 non-null object
dep_city               9999 non-null object
dep_ctry               9999 non-null object
arr_port               9999 non-null object
arr_city               9999 non-null object
arr_ctry               9999 non-null object
lst_port               9999 non-null object
lst_city               9999 non-null object
lst_ctry               9999 non-null object
brd_port               9999 non-null object
brd_city               9999 non-null object
brd_ctry       

**¿Qué representa esta tabla??**

- Esta tabla muestra las llegadas a los aeropuertos mundiales y de dónde se han producido las salidas.
- Si el tiempo que se permanece den la ciudad de destino es menor a 8 horas, se muetsra la información en una única línea aunque se registran los aeropuertos en los que se ha hecho escala.
- Se observa que la hora de 'act_date' y de 'cre_date' es siempre 00:00:00

**Plan de acción**
- Vamos a agrupar la información por 'rloc', para ver los diferentes viajes.

In [14]:
b.groupby("rloc")["act_date"].count().sort_values(ascending=False)

KeyError: 'rloc'

- Pero devuelve un **KeyError: 'rloc'**. ¿Cuál es este error?
    - Este error dice que la columna 'rloc' no existe.
    - Pero sí que existe... Entonces es que está mal escrita.
    - Como se ha comentado antes, algunas columnas tienen espacios con los que no estamos contando y 'rloc' es una de ellas.

In [15]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [16]:
b.groupby("rloc          ")["act_date           "].count().sort_values(ascending=False)

rloc          
ae15bcfc5aec0eb64b2c5204d08201d5    42
fb72a3899ed1cd353c5830388935e7f5    23
cd96f7b7fdb5743769053ba273c7eb5f    20
6b7878dd4ac59772e14ab4760ab45ad0    20
68aee71ee0f40aff44ed341f2c5f8627    16
2a86eac3e29c922f4a439fbc0480985b    15
58ee9c6852513816a39363a1621a0615    14
503d8dde8034a48c262b3f5764fc60ca    14
c9f19404e4f0755c40deaebd6e90ea84    13
182485c12b7e38aa6e2d24f7484c019b    12
cdf795b1e5710dc813c8b1147b427827    12
f3a5185d14eaa5258320bac03c5d9fc0    12
02ab3e3ded19a8cc6eb67c4413debb86    12
b8410227afd9eff71f9ed9c0e7013ddf    12
045e1c73107a2e4a39013d60e9b45aa3    12
a37584d1485cb35991e4ff1a2ba92262    12
f25cef4ed37d1483d0c4c7cfba9758ef    12
b0cd490450b69694ccafe0e08dfd821f    11
bab3d4e3fb3bee5e88340e7c359c6f13    11
2da897a9523b22f3e2cb9cd7099e0639    11
399664d8282d0587ce97c540d17eb06c    11
b0f878b0bbe51626016694032523295b    10
1e891061725828651e3e2943b79c6585    10
49edae7a10b0a8e2b2529ecc45e5dc74    10
45b87519d2fe4e85fd90512876fe9160    10
a27c351c88

- Me aparece un código de reserva repetido 42 veces.
- Parece que alguien ha estado jugando con los datos...
    - Voy a investigarlo en detenimiento filtrando por 'rloc' y ese código de reserva.

In [21]:
b[b["rloc          "]=="ae15bcfc5aec0eb64b2c5204d08201d5"].sort_values("act_date           ",ascending=False)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
8010,2013-03-30 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,ZRH,ZRH,CH,CPT,CPT,ZA,CPT,CPT,ZA,JNB,JNB,ZA,CPTJNB,CPTJNB,ZAZA,0,CPTJNB,DK,Q,Y,2013-06-09 15:10:00,2013-06-09 17:27:39,2,2013,3,NULL
8009,2013-03-30 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,CPT,CPT,ZA,LAX,LAX,US,CPTLAX,CPTLAX,USZA,1,CPTJNBFRALAX,VR,Q,Y,2013-06-09 15:10:00,2013-06-10 12:55:31,-2,2013,3,NULL
8008,2013-03-25 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,ZRH,ZRH,CH,CPT,CPT,ZA,CPTZRH,CPTZRH,CHZA,1,ZRHJNBCPT,VI,W,Y,2013-07-22 22:45:00,2013-07-23 14:17:39,2,2013,3,NULL
8007,2013-03-25 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,CPT,CPT,ZA,LAX,LAX,US,CPTLAX,CPTLAX,USZA,1,CPTJNBFRALAX,VR,Q,Y,2013-06-09 15:10:00,2013-06-10 12:55:31,2,2013,3,NULL
8006,2013-03-25 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,16396,12823,JFK,NYC,US,JNB,JNB,ZA,JNB,JNB,ZA,ZRH,ZRH,CH,JNB,JNB,ZA,JNBZRH,JNBZRH,CHZA,1,ZRHJNB,VI,W,Y,2013-07-22 22:45:00,2013-07-23 09:26:54,-2,2013,3,NULL
8005,2013-03-23 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,62781,0,JNB,JNB,ZA,LAX,LAX,US,JNB,JNB,ZA,JNB,JNB,ZA,LAX,LAX,US,JNBLAX,JNBLAX,USZA,1,JNBFRALAX,VR,Q,Y,2013-06-09 19:05:00,2013-06-10 12:55:31,-2,2013,3,NULL
8003,2013-03-22 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,62781,0,JNB,JNB,ZA,LAX,LAX,US,JNB,JNB,ZA,JNB,JNB,ZA,LAX,LAX,US,JNBLAX,JNBLAX,USZA,1,JNBFRALAX,VR,Q,Y,2013-06-09 19:05:00,2013-06-10 12:55:31,2,2013,3,NULL
8002,2013-03-22 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,ZRH,ZRH,CH,CPT,CPT,ZA,CPTZRH,CPTZRH,CHZA,1,ZRHJNBCPT,VI,W,Y,2013-07-22 22:45:00,2013-07-23 14:17:39,-2,2013,3,NULL
8001,2013-03-22 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,CPT,CPT,ZA,LAX,LAX,US,CPTLAX,CPTLAX,USZA,1,CPTJNBFRALAX,VR,Q,Y,2013-06-09 15:10:00,2013-06-10 12:55:31,-2,2013,3,NULL
8004,2013-03-22 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,62781,0,JNB,JNB,ZA,LAX,LAX,US,JNB,JNB,ZA,ZRH,ZRH,CH,JNB,JNB,ZA,JNBZRH,JNBZRH,CHZA,1,ZRHJNB,VI,W,Y,2013-07-22 22:45:00,2013-07-23 09:26:54,2,2013,3,NULL


- Si comparo el activity date y el creation date, veo que son diferentes, así que parece que este dataset muestra que falta información...
- Puedo utilizar un `describe` para obtener los estadísticos descriptivos.
    - Recordad que si pongo `describe(include=all)`, obtengo los estadísticos descriptivos no solo para los datos numéricos, sino también para las string.
    - Así puedo ver igualmente que ese código 'rloc' se repite 42 veces.

- Voy a ver si hay NaN en mi muestra de datos

In [22]:
b.isnull().

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


- Como así es imposible de analizar, voy a hacer una suma a ver si hay algún True (True=NaN)

In [23]:
b.isnull().sum()

act_date               0
source                 0
pos_ctry               0
pos_iata               0
pos_oid                0
rloc                   0
cre_date               0
duration               0
distance               0
dep_port               0
dep_city               0
dep_ctry               0
arr_port               0
arr_city               0
arr_ctry               0
lst_port               0
lst_city               0
lst_ctry               0
brd_port               0
brd_city               0
brd_ctry               0
off_port               0
off_city               0
off_ctry               0
mkt_port               0
mkt_city               0
mkt_ctry               0
intl                   0
route                  0
carrier                0
bkg_class              0
cab_class              0
brd_time               0
off_time               0
pax                    0
year                   0
month                  0
oid                    0
dtype: int64

- No hay NaN, así que puedo empezar a trabajar.
- Tengo que seleccionar las columnas de interés.
    - 'arr_port', 'pax' y 'year' son las fundamentales, como muestra el enunciado del ejercicio.
    - Voy a abarir el archivo de nuevo solo con estas columnas y decido qué hacer con los nulls.

In [23]:
import pandas as pd
pd.set_option("display.max_columns", None)
b = pd.read_csv("bookings.csv.bz2", 
                sep="^", nrows = 9999,
                usecols=["arr_port","pax","year"])

#Decidimos que hacer con los nulls o NaN. Decidimos eliminarlos en caso de que aparezcan.
b = b.dropna()

In [24]:
b.shape

(9999, 3)

In [25]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


### Comenzamos con el plan de acción:
- Calculamos el aeropuerto con más vuelos recibidos de nuestra muestra:
    - Filtramos nuestra base de datos por el año 2013.
    - Agrupamos por el código IATA de los aeropuertos.
    - Sumamos los valores acumulados
    - Pedimos que ordene la suma por 'pax' o personas en orden descendente (de mayor a menor)
    - Pedimos que nos muestre los diez primeros resultados
    - Pedimos que solo muestre la columna'pax'

In [29]:
b[b.loc[:,"year"] == 2013].groupby("arr_port").sum().sort_values("pax", ascending=False).head(10)["pax"]

arr_port
HKG         112
LGA          95
ORD          94
JFK          92
LAX          91
SFO          91
MCO          90
DCA          82
DEN          79
LHR          76
Name: pax, dtype: int64

- Sin embargo, nos ha convertido a índice el código del aeropuerto.
- Si queremos evitarlo, introducimos un `reset.index` entre medias.

In [27]:
b[b.loc[:,"year"] == 2013].groupby("arr_port").sum().sort_values("pax", ascending=False).reset_index().head(10)[["arr_port","pax"]]

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,LAX,91
5,SFO,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


## Extrapolándolo al Big Data mediante chunks
- Hemos calculado solamente los viajeros en nuestra muestra de 10.000 turistas. Sin embargo, el documento tenía más de 10M de líneas.
- Realizar el cálculo sobre el total sería muy lento, por ello, se van a utilizar los **chunks**
    - Usar **chunks** consiste en dividir el total en trozos e ir sumando resultados.
    - Los **chunks** son fundamentales para trabajar con datasets grandes.
    - Así, se dividirá el dataset en bloques de 10.000 y se irán sumando resultados hasta calcular el total.

- Recordemos el código anterior:

In [30]:
import pandas as pd
pd.set_option("display.max_columns", None)
b = pd.read_csv("bookings.csv.bz2", 
                sep="^", nrows = 9999,
                usecols=["arr_port","pax","year"])

#Vemos qué hacemos con los nulls. LOs vamos a eliminar si son nulos. AHora mismo no hace nada pero puede que en un futuro haga algo
b = b.dropna()

#Filtro por años
b=b[b["year"]==2013]
del b ["year"]

#Filtro por grupos, realizo la suma y organizo en base a pax
b_gr=b.groupby("arr_port").sum().sort_values("pax", ascending=False)

#Reseteo index. A diferencia de antes que estaba concatenado, ahora tengo que poner "inplace=True"
b_gr.reset_index(inplace=True)

#Utilizo el head
b_gr.head(10)

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,LAX,91
5,SFO,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


- Lo obtenido es un Pandas Data Frame

In [31]:
type(b)

pandas.core.frame.DataFrame

### Trabajando con chunks:

- Hay dos maneras de trabajar con chunks:
    - Una en la que no definimos el tamaño de los chunks.
    - Una en la que sí definimos el tamaño de los chunks.

#### Ejemplo 1. NO definimos el tamaño de los chunks.

    - Se añade `iterator = True` en el código.
        - Este iterator coloca el puntero y cuando hagamos head apuntará donde se ha quedado.
        - Por lo tanto, como cualquier iterador, cuidado con ejecutarlo dos veces, ya que el resultado dará 0, por estar ya al final del documento y no tener más que recorred
    - Veo que obtengo un Parser en vez de un Data Frame
    - Si utilizo `get_chunk(N)` ya sí obtengo un Data Frame

In [37]:
#Chunks
import pandas as pd
pd.set_option("display.max_columns", None)
bi = pd.read_csv("bookings.csv.bz2", 
                sep="^",
                usecols=["arr_port","pax","year"],
                nrows = 9999, iterator = True)

In [33]:
type(bi)

pandas.io.parsers.TextFileReader

In [39]:
b = bi.get_chunk(9000)

In [35]:
type(b)

pandas.core.frame.DataFrame

In [59]:
b.shape

(9000, 3)

#### Ejemplo 2. SÍ definimos el tamaño de los chunks.

    - En este caso no es necesario poner `iterator = True` en el código, ya que se asume de antemano que existe
    - EL código incluye el parámetro `chunksize = N`
        - Para comprenderlo, en el ejemplo de debajo se ve como se realizan 3 vueltas de chunksize 3000 y una de chunksize 999 hasta cubrir los 9.999 elementos de la muestra

In [66]:
#Chunks
import pandas as pd
pd.set_option("display.max_columns", None)
bi = pd.read_csv("bookings.csv.bz2", 
                sep="^",
                usecols=["arr_port","pax","year"],
                nrows = 9999, chunksize=3000, iterator = True)

for i, b in enumerate(bi):
    print(f"En la vuelta {i} el chunksize tiene un tamaño de {len(b)}")

En la vuelta 0 el chunksize tiene un tamaño de 3000
En la vuelta 1 el chunksize tiene un tamaño de 3000
En la vuelta 2 el chunksize tiene un tamaño de 3000
En la vuelta 3 el chunksize tiene un tamaño de 999


**¿Qué se va a hacer?**

- Se van a meter el código testeado con la muestra dentro del bucle y se le va a pedir que guarde los resultados en el dataframe vacío `all_chunks`
- De momento se va a eliminar el `head()` ya que no nos interesa de momento, sino más adelante

In [68]:
import pandas as pd
pd.set_option("display.max_columns", None)
bi = pd.read_csv("bookings.csv.bz2", 
                sep="^",
                usecols=["arr_port","pax","year"],
                nrows = 9999, chunksize=3000, iterator = True)

all_chunks=pd.DataFrame()

for i, b in enumerate(bi):
    print(i)
    b = b.dropna()
    b=b[b["year"]==2013]
    del b ["year"]
    b_gr=b.groupby("arr_port").sum().sort_values("pax", ascending=False).reset_index()
    all_chunks=all_chunks.append(b_gr)


0
1
2
3


- Tenemos ya todos los resultados de las 4 vueltas del bucle en `all_chunks`

In [70]:
all_chunks.shape

(1344, 2)

In [71]:
all_chunks

,arr_port,pax
0,DCA,42
1,DEN,37
2,SFO,33
3,ORD,29
4,MCO,28
5,JFK,27
6,LAX,27
7,PVG,27
8,LGA,26
9,LAS,23


- Sin embargo, no es suficiente: tenemos un "append" de cada uno de los bucles. 
    - Es decir, tenemos, por ejemplo, 4 registros de 'London Heathrow Airport' o 'LHR' cuando queremos solo uno que unifique resultados.
    - Por ello, tenemos que hacer un nuevo `groupby` con `sum()` para unificarlos

In [72]:
import pandas as pd
pd.set_option("display.max_columns", None)
bi = pd.read_csv("bookings.csv.bz2", 
                sep="^",
                usecols=["arr_port","pax","year"],
                nrows = 9999, chunksize=3000, iterator = True)

all_chunks=pd.DataFrame()
for i, b in enumerate(bi):
    print(i)
    b = b.dropna()
    b=b[b["year"]==2013]
    del b ["year"]
    b_gr=b.groupby("arr_port").sum().sort_values("pax", ascending=False).reset_index()
    all_chunks=all_chunks.append(b_gr)
    
all_result=all_chunks.groupby(["arr_port"]).sum().sort_values("pax", ascending=False).reset_index()

0
1
2
3


- Ahora ya si tenemos los resultados que queremos, pero solo de la muestra.
- Como hemos visto que los chunks funcionan, vamos a extrapolarlo al Big Data, cambiando simplemente el documento en el que trabajamos (es decir, ya no vamos a usar 'sample' sino el completo) o, en este caso, vamos a usar el mismo documento pero sin incluir las `nrows` de muestra

In [73]:
all_result.head(10)

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,LAX,91
5,SFO,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


- Para implementarlo realizo los siguientes cambios:
    - Elimino el parámetro `nrows`, ya que no quiero tomar una muestra, quiero trabajar sobre el documento completo
    - Aumento el `chunksize` para que me tome *chunks* más grandes
    - Por curiosidad, voy a añadir la función mágina `%%time` para ver cuánto tarda en evaluar.

In [100]:
%%time
import pandas as pd
pd.set_option("display.max_columns", None)
bi = pd.read_csv("bookings.csv.bz2", 
                sep="^",
                usecols=["arr_port","pax","year"],
                chunksize=1000000, iterator = True)

all_chunks=pd.DataFrame()
for i, b in enumerate(bi):
    print(i)
    b = b.dropna()
    b=b[b["year"]==2013]
    del b ["year"]
    b_gr=b.groupby("arr_port").sum().sort_values("pax", ascending=False).reset_index()
    all_chunks=all_chunks.append(b_gr)
    
all_result=all_chunks.groupby(["arr_port"]).sum().sort_values("pax", ascending=False).reset_index()

0
1
2
3
4
5
6
7
8
9
10
CPU times: user 2min 8s, sys: 466 ms, total: 2min 9s
Wall time: 2min 6s


In [101]:
all_result.head(10)

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0
5,CDG,64490.0
6,BKK,59460.0
7,MIA,58150.0
8,SFO,58000.0
9,DXB,55590.0


- El aeropuerto que recibió más visitas en 2013 fue LHR.

## BONUS.
- Utiliza Geobase/Neobase para incluir el nombre real del aeropuerto en la base de datos

In [82]:
! pip install Neobase

In [84]:
from neobase import NeoBase
geoDict = NeoBase()

- Intento averiguar qué es, pero no tengo mucha información:

In [85]:
type(geoDict)

neobase.neobase.NeoBase

- Veo que tiene como métodos `get` y `key` así que parece que esta clase es un diccionario

In [ ]:
help(geoDict)

In [87]:
dir(geoDict)

['DUPLICATES',
 'FIELDS',
 'KEY',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_distances',
 '_data',
 '_empty_value',
 'distance',
 'distance_between_locations',
 'find_closest_from_location',
 'find_near',
 'find_near_location',
 'find_with',
 'get',
 'get_location',
 'keys',
 'load',
 'set',
 'skip']

- Efectivamente, es un diccionario

In [90]:
geoDict.get("LHR").keys()

dict_keys(['__dup__', 'iata_code', 'name', 'lat', 'lng', 'page_rank', 'country_code', 'country_name', 'continent_name', 'timezone', 'city_code_list', 'city_name_list', 'location_type', 'currency'])

In [88]:
geoDict.get("LHR")

{'__dup__': set(),
 'iata_code': 'LHR',
 'name': 'London Heathrow Airport',
 'lat': '51.4775',
 'lng': '-0.461389',
 'page_rank': 0.44517643489228376,
 'country_code': 'GB',
 'country_name': 'United Kingdom',
 'continent_name': 'Europe',
 'timezone': 'Europe/London',
 'city_code_list': ['LON'],
 'city_name_list': ['London'],
 'location_type': ['A'],
 'currency': 'GBP'}

In [91]:
geoDict.get("LHR")["name"]

'London Heathrow Airport'

In [89]:
type(geoDict.get("LHR"))

dict

- Vamos a cruzarlo con *all_results*.
- Sin embargo, a partir de aquí comienzan los problemas.
- Después de mucho investigar nos podemos dar cuenta de que es porque la database `all_results` también tiene elementos en blanco en sus IATA codes de aeropuertos. Un ejemplo:

In [102]:
all_result["arr_port"][0]

'LHR     '

- Como 'LHR' es una string y tiene los métodos de las string, podemos utilizar `strip()`
- Como trabajamos con un Panda Data Frame, utilizamos la función `map`

In [105]:
all_result["arr_port"] = all_result["arr_port"].map(lambda x: x.strip())

In [107]:
all_result.head(5)

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0


- Volvemos a utilizar ahora `map`, pero esta vez para añadir el diccionario, esta vez bajo la columna "AirportName

In [108]:
all_result["AirportName"]=all_result["arr_port"].map(lambda x: geoDict.get(x)["name"])

KeyError: 'Key not found: CPQ'

- Pero da un error. No ha encontrado la Key "CPQ". Voy a verlo en profundidad

In [109]:
all_result[all_result["arr_port"]=="CPQ"]

,arr_port,pax
2257,CPQ,-20.0


- Tiene cancelaciones. Parece que es un aeropuerto que ya no existe. Lo elimino de mi listado

In [110]:
all_result = all_result[all_result["arr_port"]!="CPQ"]

- Ahora sí, vuelvo a intentarlo:

In [113]:
all_result["AirportNAme"] = all_result["arr_port"].map(lambda x: geoDict.get(x)["name"])

/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [115]:
all_result.head(10)

,arr_port,pax,AirportNAme
0,LHR,88809.0,London Heathrow Airport
1,MCO,70930.0,Orlando International Airport
2,LAX,70530.0,Los Angeles International Airport
3,LAS,69630.0,McCarran International Airport
4,JFK,66270.0,John F. Kennedy International Airport
5,CDG,64490.0,Paris Charles de Gaulle Airport
6,BKK,59460.0,Suvarnabhumi Airport
7,MIA,58150.0,Miami International Airport
8,SFO,58000.0,San Francisco International Airport
9,DXB,55590.0,Dubai International Airport


- Ya lo he conseguido, pero para tener el ejercicio de 10, veo que pax me lo devuelve en tipo `float` cuando es integer. Lo modifico:

In [116]:
all_result = all_result.astype({"pax":int})

In [117]:
all_result.head(10)

,arr_port,pax,AirportNAme
0,LHR,88809,London Heathrow Airport
1,MCO,70930,Orlando International Airport
2,LAX,70530,Los Angeles International Airport
3,LAS,69630,McCarran International Airport
4,JFK,66270,John F. Kennedy International Airport
5,CDG,64490,Paris Charles de Gaulle Airport
6,BKK,59460,Suvarnabhumi Airport
7,MIA,58150,Miami International Airport
8,SFO,58000,San Francisco International Airport
9,DXB,55590,Dubai International Airport


- Export mis resultados:

In [118]:
all_result.to_csv("top_airports.csv", sep="^", index=False)

- Compruebo que se ha guardado bien

In [120]:
ls

bookings.csv.bz2         searches.csv.bz2  Untitled1_por_si_acasov1.ipynb
bookings_sample.csv      top_airports.csv  Untitled.ipynb
bookings_sample.csv.bz2  Untitled1.ipynb


In [121]:
! head -2 top_airports.csv

arr_port^pax^AirportNAme
LHR^88809^London Heathrow Airport


## Ejercicio 5.

Vamos al ejercicio 5.
Hay que escribir un webservice

Hay que crear un JSON que es un formato texto pero con estructura de datos y sirve para intercambiar información.
DJango es una estructura de WebService pero muy compleja. SE puede usar flusk
Que es un web service? Una especie de app. Python es un webservice y se ejecuta en local (localhost), pero el web service puede ser 

In [ ]:
from flask import Flask

app = Flask("My first web service")
@app.route("/hello", methods=["GET"])
# GET    The browser tells the server to just get the information stored
def get_hello():
    return "Hello DS from the service!"

In [ ]:
app.run()

In [ ]:
Le estoy diciendo que la ruta (la de jupyter es localhost:888) yo le estoy diciento que sea hello
Y dentro de la aplicacion

In [ ]:
Así si ponemos lo que pone running on: + hello nos mostrará lo que quere,ps

In [ ]:
http://127.0.0.1:5000/hello

In [ ]:
from flask import Flask

app = Flask("My first web service")
@app.route("/hello", methods=["GET"])
# GET    The browser tells the server to just get the information stored
def get_hello():
    return "Hello DS from the service!"

#COn el codigo de debajo digo que en esa ruta estoy esperado un numero de tipo integer de nomvre n que voy a indicar abajo
@app.route("/ret_number/<int:n>", methods=["GET"])
def get_number(n):
    return "I got %d"%n

app.run()

In [ ]:
Si escribo en mi navegador: http://127.0.0.1:5000/ret_number/10

In [ ]:
Me devolverá I got 10